## 📥 Data Collection – Web Scraping Methodology

### Approach
I used **Selenium** to interact with a dynamic website and simulate a real user journey. This allowed me to:
- Apply filters directly on the website
- Navigate through multiple result pages
- Extract structured information from semi-structured listings

### Filters Applied
To focus on relevant second-hand cars, I applied the following filters:
- **Maximum price:** €15,000  
- **Minimum registration year:** 2020  
- **Vehicle types:** Saloon, Estate, Off-road / SUV  

### Data Collected
For each listing, the following fields were extracted:
- **Title Main** (brand & model)
- **Price**
- **Registration Date**
- **Mileage (KM)**
- **Power**
- **Fuel type**
- **Accident-Free indicator**

### Technical Notes
- Randomized delays were used to mimic human browsing behavior.
- Explicit waits ensured that page elements were fully loaded before interaction.
- The raw output was stored in a pandas DataFrame for further cleaning and analysis.

> ⚠️ **Note**  
> Website-specific URLs, selectors, and executable scraping code are intentionally **not included** in this repository.  
> This notebook serves as a **methodological explanation** of the data collection process.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import pandas as pd


# =========================
# CONFIGURATION (PLACEHOLDERS)
# =========================

BASE_URL = "<WEBSITE_HOME_URL>"
TOTAL_PAGES = 0  # Disabled for public version

# CSS selectors intentionally generic / anonymized
SELECTORS = {
    "cookie_accept": "<COOKIE_BUTTON_SELECTOR>",
    "max_price": "<MAX_PRICE_DROPDOWN_SELECTOR>",
    "min_year": "<MIN_YEAR_DROPDOWN_SELECTOR>",
    "vehicle_type_open": "<VEHICLE_FILTER_OPEN_SELECTOR>",
    "vehicle_type_apply": "<VEHICLE_FILTER_APPLY_SELECTOR>",
    "search_button": "<SEARCH_BUTTON_SELECTOR>",
    "listing_container": "<LISTING_ARTICLE_SELECTOR>",
    "listing_title_main": "<TITLE_MAIN_SELECTOR>",
    "listing_title_sub": "<TITLE_SUB_SELECTOR>",
    "listing_price": "<PRICE_SELECTOR>",
    "listing_details": "<DETAILS_SELECTOR>",
    "pagination_next": "<NEXT_PAGE_SELECTOR>",
}


# =========================
# DRIVER INITIALIZATION
# =========================

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    return webdriver.Chrome(options=options)


# =========================
# FILTER UTILITIES
# =========================

def safe_click(wait, selector, description="element"):
    try:
        element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
        element.click()
        time.sleep(random.uniform(1.5, 3))
        print(f"{description} clicked.")
    except:
        print(f"{description} not found.")


def apply_dropdown_filter(wait, selector, value, description):
    dropdown = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    Select(dropdown).select_by_value(value)
    time.sleep(random.uniform(1.5, 3))
    print(f"{description} set to {value}.")


# =========================
# SCRAPING LOGIC (DISABLED)
# =========================

def scrape_listings():
    raise RuntimeError(
        "Scraping disabled in public repository. "
        "This function serves as a structural template only."
    )


# =========================
# DATA PARSING LOGIC
# =========================

def parse_listing_details(details_text):
    """
    Parses semi-structured listing metadata into structured fields.
    """
    registration_date = km = power = fuel = accident_free = None

    if not details_text:
        return registration_date, km, power, fuel, accident_free

    parts = [p.strip() for p in details_text.split("•")]

    for part in parts:
        p = part.lower()
        if "accident" in p:
            accident_free = part
        elif "km" in p:
            km = part
        elif "kw" in p:
            power = part
        elif any(f in p for f in ["petrol", "diesel", "hybrid", "electric"]):
            fuel = part
        elif part[:2].upper() in ["FR", "EZ"]:
            registration_date = part

    return registration_date, km, power, fuel, accident_free


# =========================
# OUTPUT STRUCTURE
# =========================

columns = [
    "Title Main",
    "Price",
    "Registration Date",
    "KM",
    "Power",
    "Fuel",
    "Accident-Free",
]

df = pd.DataFrame(columns=columns)

print("Scraper template loaded successfully.")


Scraper template loaded successfully.


## 🎯 Data Collection – Focused Scraping (Frequent Brand–Model Pairs)

After the initial broad scrape, I ran a second data collection step to build a **more balanced and model-focused dataset**.

### Why a second step?
The first scrape gives a good overview, but some brand–model combinations appear more often than others.  
To avoid the dataset being dominated by a few random listings, I focused on cars that showed up repeatedly in the initial sample.

### What I did
1. Extracted **Brand** and **Model** from `Title Main`
2. Counted how often each **Brand-Model** pair appeared
3. Selected pairs that appeared at least **3 times**
4. For each selected pair, I:
   - Re-ran the search with the **brand + model filters applied**
   - Scraped a few result pages per pair
   - Collected the same fields as before (title, price, year, KM, power, fuel, etc.)

### Output
This produced a second dataset (`df_focused`) with the **same structure as the first one**, but with **denser coverage of the most common brand–model combinations**.

> ⚠️ **Note**  
> The executable code for this step is not included in the public repository to respect website usage policies.  
> This notebook documents the methodology only.


In [2]:
# =========================
# ORIGINAL DATA (EXAMPLE)
# =========================
# df should already exist with columns:
# ['Title Main', 'Title Sub', 'Price', 'Registration Date', 'KM', 'Power', 'Fuel', 'Accident-Free']

# =========================
# EXTRACT BRAND & MODEL (FOR FILTERING PURPOSES ONLY)
# =========================
df["Brand"] = df["Title Main"].apply(lambda x: x.split()[0] if isinstance(x, str) else None)
df["Model"] = df["Title Main"].apply(
    lambda x: x.split()[1] if isinstance(x, str) and len(x.split()) > 1 else None
)

# =========================
# IDENTIFY FREQUENT BRAND-MODEL PAIRS
# =========================
counts = df.groupby(["Brand", "Model"]).size().reset_index(name="Count")

THRESHOLD_CAR_COUNTS = None  # replace with a number, e.g., 5

frequent_pairs = counts[
    counts["Count"] >= (THRESHOLD_CAR_COUNTS or float("inf"))
][["Brand", "Model"]]

print("High-frequency brand–model combinations:")
print(frequent_pairs)

# =========================
# CONCEPTUAL DATA ENRICHMENT (DISABLED)
# =========================
"""
Original workflow:
- For each frequent Brand–Model pair:
    - Re-run filtered scraping
    - Collect additional listings
    - Append to focused dataset
"""

# Placeholder: empty DataFrame with the SAME columns as original
df_focused = pd.DataFrame(columns=[
    "Title Main",
    "Price",
    "Registration Date",
    "KM",
    "Power",
    "Fuel",
    "Accident-Free"
])

print("Focused dataset schema (template version):")
print(df_focused.columns.tolist())


High-frequency brand–model combinations:
Empty DataFrame
Columns: [Brand, Model]
Index: []
Focused dataset schema (template version):
['Title Main', 'Price', 'Registration Date', 'KM', 'Power', 'Fuel', 'Accident-Free']


In [ ]:
df_focused.to_csv('cars_data.csv', index=False)